# Structure analysis for the AIMD
This notebook is intended as a platform for developing the AIMD data analysis. The main components are the use in the MDanalysis package and DScribe

## The MDanalysis package
This is a package containing many, many useful analysis tools for structure, diffusion constants etc from AIMD simulations. Here I will just introduce some easy to use tools. Before starting, you should have MDanalysis installed. For instructions on installation etc, see [the webpage](https://userguide.mdanalysis.org/1.0.0/installation.html).

### Importing MDanalysis
First load the needed dependencies

In [ ]:
import MDAnalysis as mda
from ase import *
from ase.io import *
from MDAnalysis.analysis.base import (AnalysisBase,
                                      AnalysisFromFunction,
                                      analysis_class)
from MDAnalysis.analysis import rms
from MDAnalysis.analysis import align
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



Next load the structure and set the timestep

In [ ]:
dt = 0.02 #in ps
u = mda.Universe('AuH2O.xyz', dt=dt)
#print(u.atoms.masses)
#print('Number of frames in trajectory:', len(u.trajectory))
#print(len(u.atoms))

Next, choose the atoms you want. We are only interested in the metal atoms.

In [ ]:
atoms = u.atoms[48:]
#atoms.masses

Next, loop of over the trajectories to do some sweet, sweet analysis. 

In [ ]:
first_pos = u.trajectory[0].positions[48:].copy() # first frame positions, copy needed!
rmsds = []
for ts in u.trajectory[1:20]:
    #print(first_pos - ts.positions[48:])
    rmsds.append(rms.rmsd(first_pos, ts.positions[48:]))
times = np.arange(0,len(rmsds)*dt,dt)
#print(rmsds)
    

Next, plot the rmsds vs time.

In [ ]:
plt.plot(times,rmsds,label='RMSD')
plt.xlabel('t (ps)')
plt.ylabel('RMSD')
plt.legend()
plt.show()

# The PyScal package
While the MDanalysis is great for analyzing RMDS or correlation functions, the PyScal offers the possibility to compute order parameters. In particular, the Steinhardr order parameters are used to differentiate between the crystalline phases of materials - this is what is done herein. Have a look at the [the webpage](http://pyscal.com/en/latest/).

First load all the needed packages and structures.

In [ ]:
import pyscal.core as pc
import pyscal.crystal_structures as pcs
import numpy as np
import matplotlib.pyplot as plt


# Make the ideal FCC structures of Pt and Rh
Rh_atoms, Rh_box = pcs.make_crystal('fcc', lattice_constant=3.80, repetitions=[4,4,4])
Rh = pc.System()
Rh.atoms = Rh_atoms
Rh.box = Rh_box

Pt_atoms, Pt_box = pcs.make_crystal('bcc', lattice_constant=3.92, repetitions=[4,4,4])
Pt = pc.System()
Pt.atoms = Pt_atoms
Pt.box = Pt_box

Next, compute the radial distribution functions to choose the cutoff radii. Plot the RDFs.


In [ ]:
Rh_RDF = Rh.calculate_rdf()
Pt_RDF = Pt.calculate_rdf()
plt.plot(Rh_RDF[1], Rh_RDF[0], label='Rh')
plt.plot(Pt_RDF[1], Pt_RDF[0], label='Pt')
plt.xlabel(r'Distance ($\AA$)')
plt.ylabel('RDF')
plt.xlim([2,5])
plt.legend()
plt.show()

### Steinhardt parameters

We can choose the mid-point between the nearest and 2nd nearest neighbors as the cut-off distance. These cutoffs are then used for computing the the number of nearest neighbors and the Steinhardt parameters for both the perfect bulk and the actual clusters.

In [ ]:
Pt.find_neighbors(method='cutoff', cutoff=3.3)
Rh.find_neighbors(method='cutoff', cutoff=3.3)
print(Pt)
#Steinhardt q4 and q6
Pt.calculate_q([4,6])
Rh.calculate_q([4,6])
Pt_q = Pt.get_qvals([4, 6])
Rh_q = Rh.get_qvals([4, 6])
plt.scatter(Rh_q[0], Rh_q[1], s=60, label='Rh bulk', color='blue')

    
Pt.calculate_q([4,6])
plt.scatter(Pt_q[0], Pt_q[1], s=60, label='Pt bulk', color='orange')
plt.xlabel("$q_4$", fontsize=20)
plt.ylabel("$q_6$", fontsize=20)
plt.legend(loc=4, fontsize=15)

Having the bulk values set, lets look at the clusters and repeate the same. We loop of the structures, collect the q-values and plot with colors presenting the time.

In [ ]:
# Fetch files
from os import listdir
from os.path import isfile, join
from pyscal.traj_process import *

from ase.io import read
from matplotlib import cm
data = [f for f in listdir('Data/Pt13') if 
            f.endswith('.traj')]

read('/Users/mamimela/Desktop/andrey_60/Pt13/%s'%data[0])

cmap = 'rainbow'
# Choose the color map
colors = cm.get_cmap(cmap, len(data))
# Choose is color uniquely
color_index = np.linspace(0, 1, len(data))


for i in range(len(data)):
    # Read from using ASE
    cluster = read('/Users/mamimela/Desktop/andrey_60/Pt13/%s'%data[i])
    
    # Remove support
    cluster = cluster[216:]
    
    # Read ASE object in Pyscal
    cluster_p = read_ase(cluster)
    
    # Make into Pyscal System
    cluster_sys = pc.System()
    cluster_sys.atoms = cluster_p[0] 
    
    # Do Steinhardt
    cluster_sys.find_neighbors(method='cutoff', cutoff=3.3)
    cluster_sys.calculate_q([4,6])
    cluster_q = cluster_sys.get_qvals([4, 6])
    plt.scatter(cluster_q[0], cluster_q[1], s=10, color=colors(color_index[i]))

    
# Add also BCC and HCP points
for t, c, o in zip(['bcc','hcp','fcc'], ['black','orange','red'], [4,3,4]):
    atoms, box = pcs.make_crystal(t, lattice_constant=o, repetitions=[4,4,4])
    A = pc.System()
    A.atoms = atoms
    A.box = box
    A.find_neighbors(method='cutoff', cutoff=o)
    A.calculate_q([4,6])
    a_q = A.get_qvals([4, 6])
    plt.scatter(a_q[0], a_q[1], s=200, label=t, color=c, marker='*')
    print(c)
    
# Set colorbar
cbar = plt.colorbar(cm.ScalarMappable(cmap=cmap), ticks=[0, 1])
cbar.ax.set_yticklabels(['First', 'Last'])

plt.legend()
#Finalize plotting
plt.xlabel("$q_4$", fontsize=20)
plt.ylabel("$q_6$", fontsize=20)
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title('Pt13 Steinhardt parameters')
    
    
plt.show()


How to interpret this? The local structures resembling BCC, FCC, and HCP phases are marked in the figure. More generally, fully disoriented liquids have $q_4\approx q_6 \approx 0$. So it seems that the studied clusters in the example are not liquid-like but have some crystallinity. There's a lot of scatter in the data and the structure clearly evolves. We can also have a look at just some individual atom to see how it behavior changes in time. For instance, we can take an atom in the center of the cluster and some atom at the edge. Let's have a look...

In [ ]:
chosen_atoms = [4,12]
markers = ['o', 'P']
points = []
for i in range(len(data)):
    # Read from using ASE
    cluster = read('/Users/mamimela/Desktop/andrey_60/Pt13/%s'%data[i])
    
    # Remove support
    cluster = cluster[216:]
    
    # Read ASE object in Pyscal
    cluster_p = read_ase(cluster)
    
    # Make into Pyscal System
    cluster_sys = pc.System()
    cluster_sys.atoms = cluster_p[0] 
    
    # Do Steinhardt
    cluster_sys.find_neighbors(method='cutoff', cutoff=3.3)
    cluster_sys.calculate_q([4,6])
    cluster_q = cluster_sys.get_qvals([4, 6])
    
    for m,j in enumerate(chosen_atoms):
        if i == 0: # set labels only once. There needs to be a better way for this...
            plt.scatter(cluster_q[0][j], cluster_q[1][j], s=10, 
                    color=colors(color_index[i]), marker=markers[m],
                    label='%s'%j)
        else:
            plt.scatter(cluster_q[0][j], cluster_q[1][j], s=30, 
                    color=colors(color_index[i]), marker=markers[m])
        points.append(cluster_q[0][j])

# Add also BCC and HCP points
for t, c, o in zip(['bcc','hcp'], ['black','orange'], [4,3]):
    atoms, box = pcs.make_crystal(t, lattice_constant=o, repetitions=[4,4,4])
    A = pc.System()
    A.atoms = atoms
    A.box = box
    A.find_neighbors(method='cutoff', cutoff=o)
    A.calculate_q([4,6])
    a_q = A.get_qvals([4, 6])
    plt.scatter(a_q[0], a_q[1], s=200, label=t, color=c, marker='*')

    
# Set colorbar
cbar = plt.colorbar(cm.ScalarMappable(cmap=cmap), ticks=[0, 1])
cbar.ax.set_yticklabels(['First', 'Last'])

plt.scatter(Rh_q[0], Rh_q[1], s=200, label='fcc', color='red', marker='*')
plt.legend()
#Finalize plotting
plt.xlabel("$q_4$", fontsize=20)
plt.ylabel("$q_6$", fontsize=20)
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title('Pt13 Steinhardt parameters')

This shows how even just one atom in both the center (4) and edge (12) changes its characteristics depending on the specific geometry.

### The averaged Steinhardt parameters
Above the normal Steinhardt parameters were computed but more information can be obtained by looking at the averaged version of this parameter as shown [Lechner and Dellago](https://doi.org/10.1063/1.2977970). This extension goes beyond the first coordination sphere and hence provides more information about the local environment. To compute these averages, only slight modifications are needed.

In [ ]:
for i in range(len(data)):
    # Read from using ASE
    cluster = read('Data/Pt13/%s'%data[i])
    
    # Remove support
    cluster = cluster[216:]
    
    # Read ASE object in Pyscal
    cluster_p = read_ase(cluster)
    
    # Make into Pyscal System
    cluster_sys = pc.System()
    cluster_sys.atoms = cluster_p[0] 
    
    # Do Steinhardt
    cluster_sys.find_neighbors(method='cutoff', cutoff=3.3)
    cluster_sys.calculate_q([4,6], averaged=True)
    cluster_q = cluster_sys.get_qvals([4, 6], averaged=True)
    plt.scatter(cluster_q[0], cluster_q[1], s=10, color=colors(color_index[i]))

    
# Add also BCC and HCP points
for t, c, o in zip(['bcc','hcp','fcc'], ['black','orange','red'], [4,3,4]):
    atoms, box = pcs.make_crystal(t, lattice_constant=o, repetitions=[4,4,4])
    A = pc.System()
    A.atoms = atoms
    A.box = box
    A.find_neighbors(method='cutoff', cutoff=o)
    A.calculate_q([4,6], averaged=True )
    a_q = A.get_qvals([4, 6], averaged=True)
    plt.scatter(a_q[0], a_q[1], s=200, label=t, color=c, marker='*')
    print(c)
    
# Set colorbar
cbar = plt.colorbar(cm.ScalarMappable(cmap=cmap), ticks=[0, 1])
cbar.ax.set_yticklabels(['First', 'Last'])

plt.legend()
#Finalize plotting
plt.xlabel("$q_4$", fontsize=20)
plt.ylabel("$q_6$", fontsize=20)
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title('Pt13: Averaged Steinhardt parameters')
    
    
plt.show()

Interesting... Still $q_4\approx q_6$ but the structures are now classified as liquid-like!

# Similarity analysis
## SOAP on DScribe
To attack the question of cluster similarity, we can use the [SOAP](https://doi.org/10.1103/PhysRevB.87.184115) descriptor as implemented in the [DScribe](https://singroup.github.io/dscribe/latest/index.html) package. For details, see the documentation for the [descriptor](https://singroup.github.io/dscribe/latest/tutorials/soap.html) and [similarity kernels](https://singroup.github.io/dscribe/latest/tutorials/kernels.html).

In the first cell, the necessary modules are imported, data is read in and a bunch of stuff is set up. The idea is to do all the "general" setup here, so the SOAP/similarity analysis cells can be modular.

In [7]:
from ase.io import read, write
from dscribe.descriptors import SOAP
from dscribe.kernels import AverageKernel
from sklearn.preprocessing import normalize
import numpy as np
import os
import copy

# Trajectories to compare; for MD trajectories, the frames to compare
# are already selected here by reading only them, since it's faster
# and more convenient (no need to skip over structures later).
# The default setting also reads the very first MD frame, which should
# be a guaranteed match with some GA structure -- this is a good sanity check
structures_1 = read('Data/Pt43/pt43_gm_600_merged.traj','::1000')
structures_2 = read('Data/Pt43/pt43_andrey_60.traj', ':')

# SOAP definitions; n and l are just chosen to be "big", should be tested if
# smaller can be better here
species = ['Pt', 'Rh'] # species to include in descriptor
rcut = 12.0            # SOAP distance cutoff
nmax = 12              # number of radial basis functions
lmax = 9               # maximum degree of spherical harmonics

# set up SOAP object
soap = SOAP(
    species=species,
    periodic=True,
    rcut=rcut,
    nmax=nmax,
    lmax=lmax,
)

# set up directory for similar structure pair output
try:
    os.mkdir('Similar_structures')
except:
    print('Folder already exists')

# set up some helper functions
def is_ut(atoms, tol):
    '''Is this cell matrix upper triangular within tolerance?'''
    return abs(atoms.cell[1,0]) < tol and abs(atoms.cell[2,0]) < tol and abs(atoms.cell[2,1]) < tol

def make_ut(traj):
    '''Convert given trajectory to upper triangular cells.'''
    tol = 1e-4
    for a in traj:
        if is_ut(a, tol):
            continue
        claa = a.get_cell_lengths_and_angles()
        rot_angle = 270 - claa[5]
        a.rotate(rot_angle, 'z', rotate_cell=True)
        assert is_ut(a, tol), 'The rotated cell matrix is not upper triangular, aborted.'
        a.cell[1,0] = a.cell[2,0] = a.cell[2,1] = 0
    return traj

def remove_support(traj):
    for i in range(len(traj)):
        del traj[i][[atom.symbol == 'O' for atom in traj[i]]]
        del traj[i][[atom.symbol == 'Zr' for atom in traj[i]]]

def get_full_soaps(traj, soap, norm=True):
    soaps = []
    for frame in traj:
        soap_out = soap.create(frame)
        if norm:
            normalize(soap_out)
        soaps.append(soap_out)
    return soaps

# helper functions done

# store copies of m/zro2 structures so we can output them later and
# rotate all structures to upper triangular orientation; this makes output
# structures have matching orientations for easier inspection
structures_1_with_zro2 = make_ut(copy.deepcopy(structures_1))
structures_2_with_zro2 = make_ut(copy.deepcopy(structures_2))
    
# remove the support from the structures we actually compare
remove_support(structures_1)
remove_support(structures_2)

print('Setup done.')

Folder already exists
Setup done.


With this setup done, we can run multiple different SOAP-based methods to see which performs best. The max_dist is defined separately for each method, since it can be heavily system-, descriptor- and metric-dependent. The max_dist values have not been exhaustively tested yet, so you'll have to fiddle to find something that works nicely for you.

### Similarity kernel method
Calculate SOAPs for each frame at every atomic position and measure similarity by constructing a similarity kernel. Two options are provided for now: the average kernel and the REMatch kernel (swap by commenting the other one out). Definitions [here](https://singroup.github.io/dscribe/latest/tutorials/kernels.html).

In [10]:
max_dist = 0.003 # similarity cutoff

# get SOAP for every atomic site of every frame we've read in
# soaps_n is a list of lists of SOAP vectors
soaps_1 = get_full_soaps(structures_1, soap)
soaps_2 = get_full_soaps(structures_2, soap)

print('SOAPs done.')

# define similarity kernel object; this one calculates the similarity
# with an average kernel and a linear metric
re = AverageKernel(metric="linear")
#re = REMatchKernel(metric="linear", alpha=1, threshold=1e-6)

# run similarity analysis
for i in range(0,len(soaps_1)):
    # constantly output something while working so we can tell the binder hasn't crashed
    print('Working on element ' + str(i) + ' of soaps_1')
    for j in range(len(soaps_2)):
        dist = 1 - re.create([soaps_1[i], soaps_2[j]])[0,1]
        if dist < max_dist:
            print(i, j, dist)
            tmp=[]
            tmp.append(structures_1_with_zro2[i])
            tmp.append(structures_2_with_zro2[j])
            write('Similar_structures/'+str(i)+'_'+str(j)+'_'+str(dist)+'.traj', tmp)
            
print('Similarity analysis done.')

SOAPs done.
Working on element 0 of soaps_1
0 0 7.13458806643752e-05
0 1 0.0009148933558026373
0 3 0.0012179099694562545
0 4 0.002063901302050697
0 5 0.0023189441232464025
0 6 0.0019356792027387382
0 8 0.0001881678758043348
0 9 0.0025652439972048136
0 10 0.0007488625540671423
0 11 0.00020207928631887917
0 12 0.0005256453037932873
0 14 0.001465882383158923
0 15 0.001202977842902908
0 16 0.0002680128179440855
0 17 0.0011325425585211324
0 18 0.0017845113003623903
0 19 0.0019538760076418527
0 20 7.795636111151971e-05
0 21 0.0001125963374031258
0 22 0.000163416072692546
0 23 0.0002832274287485337
0 24 0.0001342409678682932
0 25 0.0005879153758842071
0 26 0.0004989564784889655
0 27 0.00012829618161125378
0 28 6.045527720544008e-06
0 29 0.00026589799866039154
0 30 6.294354541402924e-05
0 31 0.00026074042628054794
0 32 2.9424527575261372e-05
0 33 0.00010909238678502575
0 34 0.0014171534528567564
0 35 0.00026352943208229895
0 36 0.0003962949550685435
0 37 0.0002404597725748081
0 38 0.0001727128

### Single SOAP at centroid
This is the first SOAP attempt for posterity, only computing 1 SOAP for every structure at the center of mass and comparing SOAPs by Euclidean distance. Very fast, though possibly not very good.

In [9]:
max_dist = 0.03 # similarity cutoff

def get_soaps_at_com(traj, soap, norm=True):
    soaps = []
    for frame in traj:
        com = frame.get_center_of_mass()
        soap_out = soap.create(frame, positions=[com], n_jobs=2)
        soaps.append(soap_out)
    if norm:
        soaps = [s/np.linalg.norm(s) for s in soaps]
    return soaps

# calculate the SOAPs
soaps_1 = get_soaps_at_com(structures_1, soap, norm=True)
soaps_2 = get_soaps_at_com(structures_2, soap, norm=True)
print('SOAPs done.')

# run similarity analysis
for i in range(len(soaps_1)):
    for j in range(len(soaps_2)):
        dist = np.linalg.norm(soaps_1[i]-soaps_2[j])
        if dist < max_dist:
            print(i, j, dist)
            tmp=[]
            tmp.append(structures_1_with_zro2[i])
            tmp.append(structures_2_with_zro2[j])
            write('Similar_structures/'+str(i)+'_'+str(j)+'_'+str(dist)+'.traj', tmp)

print('Similarity analysis done.')

SOAPs done.
0 0 0.02565676080680235
0 11 0.02488877171892583
0 16 0.025747696362567043
0 21 0.02667037634068588
0 22 0.02728229235624813
0 28 0.006605579395350477
0 32 0.01958227837626082
0 33 0.0194140466827617
0 39 0.016606363396891866
Similarity analysis done.
